# Pyladies meetup <img src="https://raw.githubusercontent.com/pyladies/pyladies-assets/master/geek/png/pylady_geek_partial.png" style="display:inline" width="150"  align="right">
# 4. házi feladat megoldása

Geeky úgy döntött, hogy mielőtt bármi egyébbel tovább haladna először ezeket orvosolja, hogy később már ne okozzanak galibát. Lefuttatta eddig megírt kódjait, hogy minden a memóriában legyen és neki is állt a kódolásnak.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('bergengoc_nepszamlalas.csv', sep = ';')
df.head()

In [ ]:
df.describe(include='all')

In [ ]:
df['kor'] = 2023 - df['szuletesi_ev'] # életkor kiszámítása

df_110p = df.loc[df['kor']>=110]
print(df_110p.shape) # 1177-en töltötték már be a 110-et

df_110m = df.loc[df['kor']<110]
print(df_110m.shape) # 9349-en még 110 alatt vannak

In [ ]:
# segédtábla létrehozása
df_nepcsoportok = pd.DataFrame({'nep_kod': [0,1,2,3],
                                'nepcsoport_szoveg': ['ber', 'gen', 'goc', 'bevandorlo']})
df_nepcsoportok

<b>Adathibák javítása:</b>
- ID --> Geeky úgy döntött, a legegyszerűbben úgy tud egyedi ID-t adni a soroknak, hogy felhasználja a DataFrame index-ét, előtte azonban megnézi, hogy vannak-e teljesen megegyező sorok (tehát ahol az ID-n kívül minden más oszlop tartalma megegyezik)
- BMI --> szövegként van tárolva pedig float típusú, alakítsuk át (kezelni kell, hogy a python tizedespontot használ vessző helyett)
- Magasság --> egy list comprihension és if-else (vagy for ciklus + segédlista + if-else) segítségével kijavítja azokat a rekordokat, ahol elcsúszott a helyiérték

<b>Hiányzó adatok:</b>
  - *keresztnév*: Geeky úgy dönt ezt külön kezeli majd, így ezt oldja meg majd utoljára
  - *nem*: leggyakoribb értékkel szeretné behelyettesíteni
  - *magasság*: egészre kerekített átlaggal szeretné behelyettesíteni
  - *népcsoport*: leggyakoribb értékkel szeretné behelyettesíteni
  - *lakhely-megye*: leggyakoribb értékkel szeretné behelyettesíteni


  - Geeky úgy dönt, hogy ezek olyan problémák, amik később is előkerülhetnek, így egy függvényt szeretne írni, ami megoldja neki ezeket. A függvény bekéri az adathalmazt, az oszlop nevét, amit kezelni szeretnénk, illetve egy paramétert, ami megadja, hogy kategorikus vagy szám értékű a változó, és ennek megfelelően csinálja meg a hiányzó értékek feltöltését.
    - (pluszpontért azt is lehet kezelni a fv-en belül, hogy ha float a számértékű változó akkor float-ot helyettesítsen be, ha pedig int, akkor pedig legyen kerekítve az átlag)

In [ ]:
# Megoldás -- ID kijavítása

# Megvizsgálom, hogy hány duplikált ID van a df-ben
duplikalt_ID = df['ID'].duplicated()
print(str(df[duplikalt_ID].ID.count()) + ' db duplikált ID van')

print('------------------------------------')

# Megvizsgálom, hogy vannak-e teljesen duplikált sorok a df-ben
duplikalt_sorok = df.duplicated()  
print(str(df[duplikalt_sorok].ID.count()) + ' db duplikált sor van')

print('------------------------------------')

# Lecserélem az eredeti ID-kat sorindex_ID formátumra
df['uj_ID'] = df.index.astype(str) + '_' + df['ID'].astype(str)

# Rendezem a df-et (eldobom a régi ID oszlopot és előre hozom a uj_ID oszlopot)
df = df.drop('ID', axis=1)
df = df.reindex(columns=['uj_ID', 'keresztnev', 'szuletesi_ev', 'nem', 'magassag', 'BMI', 'nepcsoport', 'lakhely_megye', 'kor'])
df

In [ ]:
# Megoldás -- BMI

# Először is lecserélem a vesszőket tizedespontra, majd FLOAT típusú adattá alakítom
df['BMI'] = df['BMI'].str.replace(',', '.')
df['BMI'] = df['BMI'].astype(float)

print('A BMI oszlop adattípusa: ' + str(df['BMI'].dtype))
df.head()


In [ ]:
# Megoldás -- magasság

# Egy list comprehension segítségével lecserélem a 220-nál nagyobb magasság értékeket az eredeti érték tizedére
df['magassag'] = [magassag/10 if magassag > 220 else magassag for magassag in df['magassag']]

# Ellenőrzés céljából meghívok egy describe-ot a 'magassag' oszlopra, hogy lássam, tényleg nincs-e kiugró érték
df['magassag'].describe()

In [ ]:
# Megoldás -- hiányzó értékeket kezelő függvény

def hianyzo_ertek_kezelo(df, oszlop_nev, adattipus):
    
    # Ha kategorikus változó, akkor a leggyakoribb érték lesz az új érték
    if adattipus == 'categorical':
        uj_ertek = df[oszlop_nev].mode().iloc[0]
        
    # Ha szám típusú változó, akkor az átlag érték lesz az új érték (float esetén float, int esetén a kerekített érték)
    elif adattipus == 'numerical':
        if df[oszlop_nev].dtype == 'float64':
            uj_ertek = df[oszlop_nev].mean()
        elif df[oszlop_nev].dtype == 'int64':
            uj_ertek = round(df[oszlop_nev].mean())
        else:
            raise ValueError('A numerikus oszlopnak int vagy float típusúnak kell lennie')
        
    # Ha esetleg más típusú változó lenne a def-ben, akkor hibaüzenetet kap a felhasználó    
    else:
        raise ValueError('Az adattípus paraméternek "categorical" vagy "numerical" értéket kell felvennie')
    
    # Az adott oszlop NaN értékeinek feltöltése
    df[oszlop_nev] = df[oszlop_nev].fillna(uj_ertek)
    return df


In [ ]:
# Alkalmazom a függvényt a df-re
df = hianyzo_ertek_kezelo(df, 'nem', 'categorical')
df = hianyzo_ertek_kezelo(df, 'magassag', 'numerical')
df = hianyzo_ertek_kezelo(df, 'nepcsoport', 'categorical')
df = hianyzo_ertek_kezelo(df, 'lakhely_megye', 'categorical')

In [ ]:
# Megvizsgálom, hogy a 'nem', 'magassag', 'nepcsoport', 'lakhely_megye' oszlopokban maradt-e NaN érték
df[['nem', 'magassag', 'nepcsoport', 'lakhely_megye']].describe(include='all')
# mostmár 10526 érték van mind a négy oszlopban, így nem maradt egyikben sem NaN érték

### Ha utólag kényelmesebbek szeretnénk lenni és egy függvényen belül szeretnék minden oszlop módosítást kezelni, akkor így nézne ki a kód:

def hianyzo_ertek_kezelo(df, oszlop_nevek, adattipusok):
    
    # Ellenőrizzük, hogy az oszlopok és adattípusok listái ugyanolyan hosszúak-e
    if len(oszlop_nevek) != len(adattipusok):
        raise ValueError('Az oszlopok és adattípusok listáinak ugyanolyan hosszúnak kell lenniük')

    # Végigmegyünk az oszlopokon és a hozzájuk tartozó adattípusokon
    for oszlop_nev, adattipus in zip(oszlop_nevek, adattipusok):
        
        # Ha kategorikus változó, akkor a leggyakoribb érték lesz az új érték
        if adattipus == 'categorical':
            uj_ertek = df[oszlop_nev].mode().iloc[0]
            
        # Ha szám típusú változó, akkor az egészre kerekített átlag érték lesz az új érték
        elif adattipus == 'numerical':
            if df[oszlop_nev].dtype == 'float64':
                uj_ertek = df[oszlop_nev].mean()
            elif df[oszlop_nev].dtype == 'int64':
                uj_ertek = round(df[oszlop_nev].mean())
            else:
                raise ValueError('A numerikus oszlopnak int vagy float típusúnak kell lennie')
            
        # Ha esetleg más típusú változó lenne a df-ben, akkor hibaüzenetet kap a felhasználó    
        else:
            raise ValueError('Az adattípus paraméternek "categorical" vagy "numerical" értéket kell felvennie')
        
        # Az adott oszlop NaN értékeinek feltöltése
        df[oszlop_nev] = df[oszlop_nev].fillna(uj_ertek)
    
    return df

df = hianyzo_ertek_kezelo(df, ['nem', 'magassag', 'nepcsoport', 'lakhely_megye'], ['categorical', 'numerical', 'categorical', 'categorical'])


A keresztnév hiányzóinak feltöltésénél Geekynek leleményesebbnek kellett lennie, hiszen nem szeretne egy fiúnak lány nevet behelyettesíteni, vagy fordítva. Ezért először minden lehetséges 'nem' értékhez kiszedte, hogy mi az adott csoportba tartozó leggyakoribb név, hogy a feltöltést a megfelelő értékkel oldja meg.

Geeky úgy döntött az lesz a legegyszerűbb, ha az eredeti adattáblát 4 külön táblára szedi a 'nem' oszlop négy különböző értéke mentén, mindegyikre megcsinálja a behelyettesítést a táblának megfelelő leggyakoribb névvel, majd pedig újra összefűzi a táblákat, hogy visszakapja az eredetit (immáron a feltöltött keresztnevekkel.)
Geeky a sorrendre is ügyel, így a végén az ID mentén be is rendezi a táblát, hogy tényleg az eredeti formát kapja vissza.

In [ ]:
# Először is szétválasztom a df-et 4 külön df-re
df_nem_0 = df.loc[df['nem'] == 0]
df_nem_1 = df.loc[df['nem'] == 1]
df_nem_2 = df.loc[df['nem'] == 2]
df_nem_3 = df.loc[df['nem'] == 3]

In [ ]:
# Megvizsgálom melyikben mi a leggyakoribb név és az hányszor fordul elő

for i in range(4):
    df_nem = df.loc[df['nem'] == i].copy()
    
    leggyakoribb_nev = df_nem['keresztnev'].mode().iloc[0]
    elofordulasok = df_nem['keresztnev'].value_counts().iloc[0]
    
    # Lekérem azt is, melyik df-ben hány NaN érték szerepel a 'keresztnev' oszlopban a későbbi ellenőrzés céljából
    nan = df_nem['keresztnev'].isna().sum()

    print(f"Nem: {i}")
    print(f"Leggyakoribb név: {leggyakoribb_nev} - előfordulások száma: {elofordulasok}")
    print(f"NaN értékek száma: {nan}\n")


In [ ]:
# Most mindegyik df-re megcsinálom a behelyettesítést is a leggyakoribb névvel

dataframes = [df_nem_0, df_nem_1, df_nem_2, df_nem_3]

for df_nem in dataframes:
    leggyakoribb_nev = df_nem['keresztnev'].mode().iloc[0]
    df_nem['keresztnev'].fillna(leggyakoribb_nev, inplace=True)


In [ ]:
# Ellenőrzés céljából meghívok egy describe() függvényt, hogy lássam hogy alakult a top értékek freq-je
print(df_nem_0['keresztnev'].describe())
print('----------------------------------')
print(df_nem_1['keresztnev'].describe())
print('----------------------------------')
print(df_nem_2['keresztnev'].describe())
print('----------------------------------')
print(df_nem_3['keresztnev'].describe())

In [ ]:
# Összefűzöm a 4 dataframe-et újra egy nagy df-be és rendezem a sorait a new_ID oszlop szerint

df = pd.concat([df_nem_0, df_nem_1, df_nem_2, df_nem_3]).sort_values(by='uj_ID')
df


In [ ]:
# A sorba rendezéssel probléma van, hiszen az 'uj_ID' oszlop 'object' típusú adatokat tartalmaz,
# amiket nem numerikusan rendez sorba a sort_values függvény - most ezt a hibát javítom:

# Segédoszlop létrehozása az '_' előtti számok alapján
df['temp'] = df['uj_ID'].str.split('_').str[0].astype(int)

# Sorok rendezése a segédoszlop alapján
df = df.sort_values(by='temp')

# Segédoszlop eltávolítása
df = df.drop(columns='temp')

df

In [ ]:
# És végül a sok manipulálás után egy újabb describe az egész táblázatra, ellenőrzés céljából
df.describe(include='all')


Most, hogy minden adathiba és hiányzó érték a helyére került, jó lenne megfejteni, hogy mit is jelentenek a kódok a nem, illetve népcsoport oszlopoknál. Mivel a népcsoporthoz tartozó mappinget már megkapta, így már csak a 'nem' megfeleltetésekre volt szüksége, így elkérte a hivatalnoktól. Az alábbi információt kapta:
  - 0 - nő
  - 1 - férfi
  - 2 - egyéb
  - 3 - nem kíván válaszolni

Geeky létrehozta ezt a segédtáblát is, majd pedig két join (merge) művelettel hozzákapcsolta a táblázathoz, hogy egyértelműek legyenek ezek a mezők is. A megfelelő összekapcsolás után a kódokat tartalmazó oszlopokat eldobta.

In [ ]:
df.head()

In [ ]:
# Megoldás - 'nem' segédtábla létrehozása
df_nem = pd.DataFrame({'nem_kod': [0,1,2,3],
                       'nem_szoveg': ['no', 'ferfi', 'egyeb', 'nincs_valasz']})
df_nem

In [ ]:
# Megoldás - 'nem' leképzés csatolása (merge)
df = df.merge(df_nem, how = 'left', left_on='nem', right_on='nem_kod')
df

In [ ]:
# Megoldás - népcsoport leképzés csatolása (merge)
df = df.merge(df_nepcsoportok, how = 'left', left_on='nepcsoport', right_on='nep_kod')
df

In [ ]:
# Megoldás - felesleges oszlopok eldobása, új oszlopok átnevezése és az oszlopok újra rendezése
df = df.drop(['nem', 'nepcsoport', 'nem_kod', 'nep_kod'], axis=1)
df = df.rename(columns={'nem_szoveg': 'nem', 'nepcsoport_szoveg': 'nepcsoport'})
df = df.reindex(columns=['uj_ID', 'keresztnev', 'szuletesi_ev', 'nem', 'magassag', 'BMI', 'nepcsoport', 'lakhely_megye', 'kor'])
df

#### Értem, hogy a merge() függvény alkalmazását szerettük volna begyakorolni, de magamtól egyébként inkább a replace()-es megoldás jutott volna eszembe:

   - df['nepcsoport'] = df['nepcsoport'].replace({0: 'ber', 1: 'gen', 2: 'goc', 3:'bevandorlo'})
   - df['nem'] = df['nem'].replace({0: 'no', 1: 'ferfi', 2: 'egyeb', 3:'nincs_valasz'})


Most, hogy végre teljesen elemzésre alkalmas formára hozta Geeky a táblázatot, megnézte, hogy milyen információkra kíváncsi a hivatalnok.

- az egyes népcsoportokkal kapcsolatban:
  - hány lakos tartozik oda (count)
  - min-átlag-max életkor
  - átlag-medián magasság és BMI
- az egyes megyékkel kapcsolatban:
  - hány lakos tartozik oda (count)
  - átlag életkor
  - medián magasság és BMI
  - hány különböző népcsoport él ott (nunique)
- ki a legidősebb lakosunk? (és mik az adatai?) (idxmax segíthet, de más megoldás is jó)

A hivatalnok látja, hogy Geeky épp a kimutatások részleteit böngészi, így hozzáteszi:
- ezek a legsürgetőbb információk amikre szükségünk van, a többivel meg tudjuk várni az elemzőnket. Nagyon könnyű dolga lesz azok után, hogy ilyen szépen előkészítette az adatokat. De ha van kedve, akkor egy jegeskávé kuponért (plusz pontért) cserébe lepjen meg minket, hátha eszébe jut valami olyan nézet, ami nekünk nem, és ezzel feldobhatjuk a minisztériumnak adott jelentésünket.

Geeky először exportálta az előkészített adatot egy csv file-ba, hogy meg tudja osztani az előkészített adatokat, majd gyorsan lekódolta a kimutatásokat is.


In [ ]:
# Megoldás - adatok exportálása
df.to_csv('bergengoc_nepszamlalas_tisztitott.csv', index = False)

### Dolgozzunk mostantól a tisztított .csv-vel

In [ ]:
df = pd.read_csv('bergengoc_nepszamlalas_tisztitott.csv', sep = ',')
df

In [ ]:
# Megoldás - népcsoport szerinti kimutatás

df_nepcsoport = df.groupby(['nepcsoport'], as_index = False).agg({'uj_ID': ['count'],
                                                                  'kor': ['min', 'mean', 'max'],
                                                                  'magassag': ['mean', 'median'],
                                                                  'BMI': ['mean', 'median']})
new_cols = ['nepcsoport'] + [i[0] + '_' + i[1] for i in df_nepcsoport.columns[1:]]
df_nepcsoport.columns = new_cols
df_nepcsoport


In [ ]:
# Megoldás - lakhely szerinti kimutatás

df_megye = df.groupby(['lakhely_megye'], as_index = False).agg({'uj_ID': ['count'],
                                                                'kor': ['mean'],
                                                                'magassag': ['median'],
                                                                'BMI': ['median'],
                                                                'nepcsoport': ['nunique']})
new_columns = ['lakhely_megye'] + [column[0] + '_' + column[1] for column in df_megye.columns[1:]]
df_megye.columns = new_columns
df_megye

In [ ]:
# Megoldás - legidősebb bergengóc

# Először is megvizsgálom, hogy 1 vagy több "legidősebb" korú lakos van-e az adathalmazban

max_eletkor = df['kor'].max()
max_eletkor_count = df['kor'].value_counts().loc[df['kor'].max()]

if max_eletkor_count > 1:
    print(f'Van {max_eletkor_count} "legidősebb" korú lakos az adathalmazban, akik mind {max_eletkor} évesek.')
else:
    print("Csak egy legidősebb korú lakos van az adathalmazban, aki {max_eletkor} éves.")


In [ ]:
# Leszűröm a df-t a 123 éves lakosokra

df_legidosebb_lakosok = df[df.kor == 123]
df_legidosebb_lakosok

Geeky, miután mindent is megcsinált, egyszerűen annyira belejött a kódolásba, hogy úgy döntött még ír 2 függvényt:
- az egyik a BMI és a magasság alapján kiszámolja a súlyt (bemeneti érték a magasság (cm) és a BMI)
  - súly = BMI * magasság(m)^2
- a másikkal pedig könnyen lehetővé teszi, hogy egy adatbázisban szűrni lehessen:
  - bemeneti érték a dataframe (adat) és a *kwargs*
  - a függvény pedig megnézi, hogy a *kwargs* részben megadott paraméterek közül melyek vannak benne az adatbázis oszlopai között, és annak megfelelően leszűri az adatokat, és visszatér a szűrt adatokkal
    - magát az adatszűrést ennek mentén érdemes megírni, ahol az *adat* a függvény bemeneti paramétere, a *lista* és *feltétel* részeket pedig a *kwargs* paraméterek alapján kell megírni
 ```
for i in lista:
        adat = adat.loc[feltétel]
```
    - tehát pl megadjuk hogy magassag = 100, kor = 25, iskolai_vegzettseg = 'altalanos' és a függvény visszatér egy olyan dataframmel, ami szűrve van arra, hogy a magassag=100 és a kor = 25 (és nem akad fenn azon, hogy nincs iskolai végzettség az oszlopok között)





In [ ]:
df.describe(include='all')

In [ ]:
# Megoldás - súly kiszámító függvény

def suly_kalkulator(magassag, BMI):
    # Ellenőrzi, hogy a bemeneti értékek számok-e
    if ((type(magassag) not in [int, float]) or (type(BMI) not in [int, float])):
        raise ValueError("A magasság és a BMI értékeknek számoknak kell lenniük!")

    # Ellenőrzi a magasság értékét
    if magassag <= 70 or magassag > 250:
        raise ValueError("A magasság értékének 70 cm és 250 cm között kell lennie!")

    # Ellenőrzi a BMI értékét
    if BMI <= 10 or BMI > 50:
        raise ValueError("A BMI értékének 10 és 50 között kell lennie!")
        
    # Ha minden rendben érték rendben van, akkor elvégzi a számítást
    suly = BMI * (magassag / 100)**2
    suly_kerekitve = round(suly, 2)
    
    #Végül kiírja az eredményt
    print(f"A megadott adatok alapján a személy súlya {suly_kerekitve} kg")


In [ ]:
# Példa hívás a suly_kalkulatorhoz
suly_kalkulator(160, 17.97)


Adatszűrés

In [ ]:
# Megoldás - adatbázis szűrés számítás

def df_szuro(dataframe, **kwargs):
    
    #Létrehoz egy másolatot az eredeti adatbázisról, hogy az eredeti adatok ne változzanak
    szurt_df = df.copy()
    
    for kulcs, ertek in kwargs.items():
        
        #Ellenőrzi, hogy az adott kulcsszó megtalálható-e az adatbázis oszlopaiban...
        if kulcs in szurt_df.columns:
            #...és ha igen, akkor leszűri a df-et az adott {kulcs:érték} pár alapján
            szurt_df = szurt_df[szurt_df[kulcs] == ertek]
            
    return szurt_df


In [ ]:
df_szuro(df, szuletesi_ev=2023, nem='no', nepcsoport='ber', szeme_szine='barna')

#### Bár kifejezetten nem kérte a feladat, de azért ebbe a függvénybe is raknék ValueError-okat, ha rajtam múlna

    def df_szuro(dataframe, **kwargs):

        #Ellenőrizzük, hogy az adat DataFrame-e
        if type(df) != pd.DataFrame:
        raise ValueError("A bemeneti adatnak pandas DataFrame-nek kell lennie!")

        #Létrehoz egy másolatot az eredeti adatbázisról, hogy az eredeti adatok ne változzanak
        szurt_df = df.copy()

        for kulcs, ertek in kwargs.items():

            # Ellenőrizzük, hogy az adott oszlop létezik-e
            if kulcs not in szurt_df.columns:
                raise ValueError(f"A(z) '{kulcs}' kulcsszó nem létező oszlopnév!")

            # Ellenőrizzük, hogy az oszlop tartalmazza-e az adott értéket
            if ertek not in szurt_df[kulcs].unique():
                raise ValueError(f"A(z) '{kulcs}' oszlopban nincs '{ertek}' érték!")

            #Ellenőrzi, hogy az adott kulcsszó megtalálható-e az adatbázis oszlopaiban...
            if kulcs in szurt_df.columns:
                #...és ha igen, akkor leszűri a df-et az adott {kulcs:érték} pár alapján
                szurt_df = szurt_df[szurt_df[kulcs] == ertek]

        return szurt_df


Geeky, habár nagyon belejött a kódolásba, hirtelen realizálta, hogy mennyi az idő, így sebtiben elküldte a kódot és az előkészített adatot a hivatalnoknak, majd hátára kapta hátizsákját és elindult Bergenburg meghódítására.